In [6]:
import setup
setup.init_django()

In [7]:
from market.models import StockQuote

In [8]:
from django.db.models import Avg, F, RowRange, Window
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [9]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

5250

In [10]:
total = 0
for obj in qs:
    total += obj.close_price

In [11]:
total / qs.count()

Decimal('242.7401499619047619047619048')

In [12]:
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 3, 'avg_price': Decimal('421.9201978461538462')}, {'company': 5, 'avg_price': Decimal('24.8383880952380952')}, {'company': 4, 'avg_price': Decimal('568.5540252396166134')}, {'company': 2, 'avg_price': Decimal('176.2666191823899371')}, {'company': 1, 'avg_price': Decimal('228.2590953779366701')}]>

In [13]:
count = 5
ticker = "AAPL"
rolling_qs = list(qs.filter(company__ticker=ticker).order_by('-time')[:count])
rolling_qs.reverse()

In [14]:
rolling_qs

[<StockQuote: StockQuote object (64597)>,
 <StockQuote: StockQuote object (64598)>,
 <StockQuote: StockQuote object (64599)>,
 <StockQuote: StockQuote object (64600)>,
 <StockQuote: StockQuote object (64601)>]

In [15]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

1 64597 225.1500 225.1500
2 64598 225.2000 225.1750
3 64599 225.2000 225.1833333333333333333333333
4 64600 225.1700 225.1800
5 64601 225.2600 225.1960


In [11]:
qs = StockQuote.objects.filter(company__ticker=ticker, time__range=(start_date, end_date))

In [12]:
frame_start = -(count - 1)
ma_val = qs.annotate(
        ma=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=frame_start, end=0),
        )
).order_by('-time')

In [13]:
for obj in ma_val[:5]:
    print(obj.id, obj.close_price, obj.ma, obj.time)

6466361 225.9900 226.1517400000000000 2024-11-13 18:40:00+00:00
6466360 225.9750 226.2457400000000000 2024-11-13 18:35:00+00:00
6466359 225.9787 226.3627400000000000 2024-11-13 18:30:00+00:00
6466358 226.3638 226.4130000000000000 2024-11-13 18:25:00+00:00
6466357 226.4512 226.4692400000000000 2024-11-13 18:20:00+00:00


In [14]:
frame_start = -(count - 1)
ma_vals = qs.annotate(
        ma_5=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=-4, end=0),
        ),
        ma_20=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=-19, end=0),
        )
).order_by('-time').first()

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20

(6466361,
 Decimal('225.9900'),
 Decimal('226.1517400000000000'),
 Decimal('225.9005650000000000'))